In [1]:
import os
import sys

from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID
from whoosh.qparser import QueryParser, OrGroup
from whoosh import scoring
from whoosh.index import open_dir
# from whoosh.lang.porter import stem
import snowballstemmer

import pandas as pd

Die Stichwortsuche des Sphinx Framework soll als Baseline für die getesteten Sprachmodelle dienen. Diese ist jedoch so nicht leicht portiertbar, da sie in Javascript verfasst und auf Dependencies wie JQuery aufbaut. Weitaus einfacher ist die Implementierung einer ähnlichen Search-Enginge mit **whoosh**, die aus vier Elementen besteht:

- Stemmer (Reduktion der Wörter in Dokumenten und Anfragen auf Wortstamm)
- Herausfiltern von Stoppwörtern aus Suchanfragen
- Suche nach Stichwörtern in Dokumenten
- Ranking von Dokumenten nach Relevanz nach erfolgreichem Auffinden von Stichwörtern

Für den Stemmer wird **snowball** genutzt (wie auch in der Sphinx Search Engine)

Überprüfen, ob Stemming auch auf deutschen Wörtern funktioniert

In [2]:
stemmer = snowballstemmer.stemmer('german')
print(stemmer.stemWords("Viele Schritte".split()))

['Viel', 'Schritt']


Diese Liste an deutschen Stopp-Wörtern ist der Doku selbst entnommen.

In [3]:
stopwords = ["aber","alle","allem","allen","aller","alles","als","also","am","an","ander","andere","anderem","anderen","anderer","anderes","anderm","andern","anderr","anders","auch","auf","aus","bei","bin","bis","bist","da","damit","dann","das","dasselbe","dazu","da\u00df","dein","deine","deinem","deinen","deiner","deines","dem","demselben","den","denn","denselben","der","derer","derselbe","derselben","des","desselben","dessen","dich","die","dies","diese","dieselbe","dieselben","diesem","diesen","dieser","dieses","dir","doch","dort","du","durch","ein","eine","einem","einen","einer","eines","einig","einige","einigem","einigen","einiger","einiges","einmal","er","es","etwas","euch","euer","eure","eurem","euren","eurer","eures","f\u00fcr","gegen","gewesen","hab","habe","haben","hat","hatte","hatten","hier","hin","hinter","ich","ihm","ihn","ihnen","ihr","ihre","ihrem","ihren","ihrer","ihres","im","in","indem","ins","ist","jede","jedem","jeden","jeder","jedes","jene","jenem","jenen","jener","jenes","jetzt","kann","kein","keine","keinem","keinen","keiner","keines","k\u00f6nnen","k\u00f6nnte","machen","man","manche","manchem","manchen","mancher","manches","mein","meine","meinem","meinen","meiner","meines","mich","mir","mit","muss","musste","nach","nicht","nichts","noch","nun","nur","ob","oder","ohne","sehr","sein","seine","seinem","seinen","seiner","seines","selbst","sich","sie","sind","so","solche","solchem","solchen","solcher","solches","soll","sollte","sondern","sonst","um","und","uns","unse","unsem","unsen","unser","unses","unter","viel","vom","von","vor","war","waren","warst","was","weg","weil","weiter","welche","welchem","welchen","welcher","welches","wenn","werde","werden","wie","wieder","will","wir","wird","wirst","wo","wollen","wollte","w\u00e4hrend","w\u00fcrde","w\u00fcrden","zu","zum","zur","zwar","zwischen","\u00fcber"];

In [4]:
len(stopwords)

231

Anfragen sind *case sensitive*, also muss alles *to lower* konvertiert werden.

In [5]:
"was" in stopwords

True

In [6]:
"Was" in stopwords

False

In [7]:
def prepare_string(string, stopwords):
   
    # replace question mark
    query_str = string.replace('?', '')
    
    # stem words
    query_tokens = stemmer.stemWords(query_str.split())
    
    # remove stopwords and convert to lower
    query_filter = [token.lower() for token in query_tokens if token.lower() not in stopwords]

    # re-assemble string
    return ' '.join(e for e in query_filter)    

Basis für die Suche ist das Bauen eindes Indexes aus allen Dokumenten

In [8]:
def createSearchableData(documents, indexdir_path, title_col=None, path_col=None, text_col=None, ending='.html'):   
 
    '''
    Schema definition: title(name of file), path(as ID), content(indexed
    but not stored),textdata (stored text content)
    '''
    
    # create Schema for indexing documents
    schema = Schema(
        title    = TEXT(stored=True),
        path     = ID(stored=True), 
        content  = TEXT,
        textdata = TEXT(stored=True)
    )
    
    # create dir for storing indexing results
    if not os.path.exists(indexdir_path):
        print("writing index to:", indexdir_path)
        os.mkdir(indexdir_path)
 
    # Creating a index writer to add document as per schema
    ix = create_in(indexdir_path, schema)
    writer = ix.writer()
 
    # add every document to index
    for index, row in documents.iterrows():

        path = row[path_col] if path_col else None
        title = row[title_col] if title_col else None
        text = prepare_string(row[text_col], stopwords)

        writer.add_document(title=title, path=path, content=text, textdata=text)
    
    writer.commit()

### Erstellen des Suchindex aus Dokumenten

In [9]:
documents = catalog.load("ecu_test_doku_parsed")

2022-05-12 13:03:40,072 - kedro.io.data_catalog - INFO - Loading data from `ecu_test_doku_parsed` (CSVDataSet)...


In [10]:
documents.head(1)

,filename,title,sub_topics,body,links,imgs
0,Bedienung_EasyInsert.html,EasyInsert,['Tastenkombinationen\uf0c1'],EasyInsert — ECU-TEST 2022.2 Anwenderhandbuch ...,"['../index.html', '../Getting_Started/GettingS...","['../_static/logo.png', '../_images/EasyInsert..."


In [11]:
indexdir = "../data/03_primary/ecu_test_index/"
createSearchableData(documents, indexdir, title_col='title', path_col='filename', text_col='body')

### Testen der Suchfunktion und Ausgabe gefundener Resultate in DataFrame

In [12]:
def query(query_str, parser='all', topN_results=10, verbose=True):
    
    result_rows = []   

    query_str = prepare_string(query_str, stopwords)
    if verbose: print(f'the query is: "{query_str}"')

    ix = open_dir(indexdir) 
    with ix.searcher(weighting=scoring.Frequency) as searcher:
        if parser == 'all':
            query = QueryParser("content", ix.schema).parse(query_str)
        elif parser == 'any':
            query = QueryParser("content", ix.schema, group=OrGroup).parse(query_str)

        results = searcher.search(query)

        if len(results) > 0:
            if verbose: print(f'\nsearch yielded {len(results)} results in total')
            num_res = topN_results if topN_results <= len(results) else len(results)
            for i in range(num_res):
                # result_rows.append([results[i].score, results[i]['title'], results[i]['path']])
                result_rows.append([results[i].score, results[i]['path']])
        else:
            if verbose: print('search yielded no hits')
            return []

    # convert results to dataframe and display
    df = pd.DataFrame(result_rows)
    # df.columns = ['Score', 'Title', 'Path']
    df.columns = ['Score', 'Path']
    
    return df

### Testquery 1

In [13]:
query("Was ist ein Traceschritt?")

the query is: "traceschritt"

search yielded 34 results in total


,Score,Path
0,25.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Gene...
1,21.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Trac...
2,15.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Sign...
3,14.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Bere...
4,12.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Asse...
5,10.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Konz...
6,9.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Abla...
7,8.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Diag...
8,7.0,Bedienung_Tastenkombinationen.html
9,6.0,TRACE-CHECK_Handbuch_Traceschrittvorlagen_Arra...


### Testquery 2

In [14]:
query("Wie erstellt man eine Traceschritt Analyse?")

the query is: "erstellt traceschritt analys"

search yielded 15 results in total


,Score,Path
0,31.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Gene...
1,22.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Bere...
2,18.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Konz...
3,13.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Diag...
4,12.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Abla...
5,12.0,TRACE-CHECK_Tutorial_Erste_Traceanalyse_Beispi...
6,11.0,TRACE-CHECK_Handbuch_Traceschrittvorlagen_Arra...
7,11.0,TRACE-CHECK_Referenz_Referenz_Timing-Diagramme...
8,11.0,TRACE-CHECK_Tutorial_Erste_Traceanalyse_Beispi...
9,10.0,Hauptprogramm_Aktionsfenster_Testschritte-Kart...


### Testquery 3

In [15]:
query("Wie wird ein Testfall erstellt?")

the query is: "testfall erstellt"

search yielded 57 results in total


,Score,Path
0,32.0,Testausfuehrung_Testausfuehrung.html
1,21.0,Getting_Started_GettingStarted.html
2,15.0,Testausfuehrung_Analyse-Jobs.html
3,14.0,Hauptprogramm_Editor_Projekteditor.html
4,13.0,Einfuehrung_Testen.html
5,9.0,Einfuehrung_Grundlagen_Mappingkonzept.html
6,9.0,Einfuehrung_Grundlagen_Zeitkonzept.html
7,8.0,Hauptprogramm_Aktionsfenster_WORKSPACE-Karte.html
8,8.0,Hauptprogramm_Reportviewer.html
9,7.0,Einfuehrung_Erster_Start_Erstellen_Package.html


### Testquery 4

In [16]:
query("Was ist der Unterschied zwischen Testfall und Traceschritt?", parser='all')

the query is: "unterschied zwisch testfall traceschritt"

search yielded 2 results in total


,Score,Path
0,22.0,Testausfuehrung_Analyse-Jobs.html
1,7.0,TRACE-CHECK_Handbuch_Traceanalyse_im_Projekt_S...


In [17]:
query("Was ist der Unterschied zwischen Testfall und Traceschritt?", parser='any')

the query is: "unterschied zwisch testfall traceschritt"

search yielded 257 results in total


,Score,Path
0,32.0,Testausfuehrung_Testausfuehrung.html
1,28.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Gene...
2,23.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Bere...
3,23.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Trac...
4,22.0,Testausfuehrung_Analyse-Jobs.html
5,18.0,Testausfuehrung_Interaktives_Testen.html
6,16.0,Hauptprogramm_Aktionsfenster_Testschritte-Kart...
7,16.0,TRACE-CHECK_Handbuch_Traceanalyse-Entwurf_Sign...
8,15.0,Bedienung_Tastenkombinationen.html
9,15.0,TRACE-CHECK_Referenz_Referenz_Timing-Diagramme...


## Testen der Suchfunktion an Germanquad Datensatz

### Laden des Datensatzes als DataFrame

In [18]:
germanquad = catalog.load("germanquad_validation")

2022-05-12 13:04:12,969 - kedro.io.data_catalog - INFO - Loading data from `germanquad_validation` (CSVDataSet)...


In [19]:
germanquad.head()

,id,context,question,answers
0,40369,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...,Was kann den Verschleiß des seillosen Aufzuges...,{'text': array(['elektromagnetischer Linearfüh...
1,40370,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...,In welcher deutschen Stadt wird der seillose A...,"{'text': array(['Rottweil', 'Rottweil'], dtype..."
2,40366,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...,Wo wurde ein seilloser Aufzug entwickelt?,{'text': array(['An der RWTH Aachen im Institu...
3,40367,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...,Wie funktioniert ein seilloser Aufzug?,{'text': array(['Die Kabine wird hierbei durch...
4,40368,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...,Wann muss man die Zieletage in seillosen Aufzü...,{'text': array(['vor Fahrtantritt (d.\xa0h. no...


### Hashen aller Kontexte, da Kontexte mehrfach auftreten (mehrere Fragen pro Kontext)

In [20]:
import hashlib
import base64

def hash_context(text):
    return base64.b64encode(hashlib.md5(text.encode('utf-8')).digest()).decode()

In [21]:
_test_context = germanquad['context'].iloc[1]
_test_context[:100]

'Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn der RWTH Aachen im Institut für Elektrische Maschinen wur'

In [22]:
hsh = hashlib.md5(_test_context.encode('utf-8')).digest()
base64.b64encode(hsh).decode()

'1Hu25PJK/VZwj72Is7G8vw=='

In [23]:
hash_context(_test_context)

'1Hu25PJK/VZwj72Is7G8vw=='

### Erstellen eines Index für Germanquad
Dabei werden nur unique Kontexte indiziert. Der Inhalt (Text) wird gehashed und dient als Pfad um später beim Testen einen Fragekontext einem Kontext im Index zuordnen zu können.

In [24]:
germanquad_contexts = germanquad['context']

In [25]:
germanquad_contexts.unique().shape

(474,)

In [26]:
unique_contexts_with_hash = []
for c in germanquad_contexts.unique():
    hsh = hash_context(c)
    unique_contexts_with_hash.append({"hash": hsh, "context": c})

In [27]:
pd.DataFrame(unique_contexts_with_hash).head(5)

,hash,context
0,1Hu25PJK/VZwj72Is7G8vw==,Aufzugsanlage\n\n=== Seilloser Aufzug ===\nAn ...
1,60+PszMaTC1g1qvjxof4ng==,Sichuan\n\n=== Landwirtschaft ===\nSichuan gil...
2,BNjqad/eGjINJp6eKBXS1g==,Antenne\n\n==== Polarisation ====\nAntennen st...
3,pfO/sDJuLD9rxUNBLlzYVQ==,Softwaretest\n\n== Ziele ==\n''Globales Ziel''...
4,FN6vYWPIl+LT4/A3IfIEqg==,Tuberkulose\n\n=== 19. Jahrhundert ===\nAufgru...


In [28]:
indexdir = "../data/03_primary/germanquad_index/"
createSearchableData(pd.DataFrame(unique_contexts_with_hash), indexdir, path_col='hash', text_col='context')

### Zufällige Frage wählen und prüfen ob richtiger Kontext von Stichwortsuche gefunden wird

In [69]:
# pick random sample and query the question
s = germanquad.sample()

s_q = s['question'].iloc[0]
s_id = s['id'].iloc[0]
print("question is :", s_q)

# calculate results
result = query(s_q)

# check wether hash of context - question pair is within top 5 answers
correct_context = hash_context(s['context'].iloc[0])
if len(result) > 0:
    print(f'''context is: {correct_context} and is 
        {'not ' if correct_context not in list(result['Path'])[:5] else ''}among top 5 answers''')
    
display(result)

question is : Welchen Platz belegte Arnold Schwarzenegger bei seinem ersten IFBB Meisterschaft?
the query is: "welch platz belegt arnold schwarzenegg erst ifbb meisterschaft"

search yielded 1 results in total
context is: ZvwEmZZolKvfidgl5SGi+A== and is 
        among top 5 answers


,Score,Path
0,47.0,ZvwEmZZolKvfidgl5SGi+A==


### Berechne Top5-Score für alle Fragen + Kontexte
Frage gilt dann als richtig beantwortet, wenn Kontext zur Frage innerhalb der ersten 5 Suchergebnisse ist. Dabei müssen **alle** Tokens der Query (nach Filtern der Suchanfrage) im Kontext enthalten sein (Suchtyp: all)

In [30]:
hit_count = 0
for index, row in germanquad.iterrows():
    _q = row['question']
    _id = row['id']
    _res = query(_q, verbose=False, parser='all')
    _corr_ctxt = hash_context(row['context'])
    if len(_res) == 0: continue
    if _corr_ctxt in list(_res['Path'])[:5]:
        hit_count += 1      

In [31]:
print('accuracy is:', round(hit_count / len(germanquad), 2))

accuracy is: 0.12


### Berechnung Top5-Score mit Suchtyp: 'any'
Anstelle von allen Wörtern (Wortstämmen) der Suchanfragen muss jetzt nur **eins** der Wörter im Kontext enthalten sein, um gefunden zu werden. 

In [32]:
hit_count = 0
for index, row in germanquad.iterrows():
    _q = row['question']
    _id = row['id']
    _res = query(_q, verbose=False, parser='any')
    _corr_ctxt = hash_context(row['context'])
    if len(_res) == 0: continue
    if _corr_ctxt in list(_res['Path'])[:5]:
        hit_count += 1      

In [33]:
print('accuracy is:', round(hit_count / len(germanquad), 2))

accuracy is: 0.77


### Berechnung Top1-Score mit Suchtyp: 'any'

In [34]:
hit_count = 0
for index, row in germanquad.iterrows():
    _q = row['question']
    _id = row['id']
    _res = query(_q, verbose=False, parser='any')
    _corr_ctxt = hash_context(row['context'])
    if len(_res) == 0: continue
    if _corr_ctxt in list(_res['Path'])[:1]:
        hit_count += 1      

In [35]:
print('accuracy is:', round(hit_count / len(germanquad), 2))

accuracy is: 0.48
